# NEURAL NETWORKS

In [ ]:
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn import svm
import pylab as pl
import numpy as np
import pandas as pd  
from sklearn.model_selection import train_test_split

C:\Users\erika\Anac\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#read the data
df = pd.read_csv('./data/joinedData.csv', sep=r'\s*(?:\||\#|\,)\s*',
                 engine='python')

#change the classes to numbers
Class = {'LUAD': 0,'BRCA': 1,'KIRC': 2,'PRAD': 3,'COAD': 4} 
df.Class = [Class[item] for item in df.Class] 
df = df.drop('Unnamed: 0',1)
df = df.drop('Unnamed: 0.1',1)
df

#separate the data into X and y
X = df.drop('Class', axis=1).values
y = df['Class'].values
y = np.asarray(y)

#standardize the data
X = (X - X.mean()) / (X.max() - X.min())

#Split the data set to test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)  
X_train

array([[-0.31009073, -0.13952523, -0.11792117, ...,  0.14745323,
         0.08970348, -0.31009073],
       [-0.31009073, -0.1269397 , -0.19466531, ...,  0.17132615,
        -0.09676468, -0.29304327],
       [-0.31009073, -0.1250641 , -0.15804081, ...,  0.12718386,
        -0.11644316, -0.28899749],
       ...,
       [-0.31009073, -0.1179661 , -0.17665057, ...,  0.18994972,
         0.0666543 , -0.31009073],
       [-0.31009073, -0.04898057, -0.05965011, ...,  0.14922751,
         0.17264622, -0.31009073],
       [-0.31009073, -0.10308701, -0.11619049, ...,  0.19053238,
        -0.10996971, -0.31009073]])

In [3]:
#get the estimator
estimator = tf.estimator.DNNClassifier(
    feature_columns=[tf.feature_column.numeric_column('x', shape=X_train.shape[1:])],
    hidden_units=[1000, 500, 250], 
    optimizer=tf.train.ProximalAdagradOptimizer(
      learning_rate=0.01,
      l1_regularization_strength=0.001
    ), #optimizer was used to improve the estimator
    n_classes=5) #the number of label classes, we have 5

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\erika\\AppData\\Local\\Temp\\tmpc8xmdbyk', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000021C6F2F8DD8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [4]:
# defining the training inputs
train = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=X_test.shape[0],
    num_epochs=None,
    shuffle=False,
    num_threads=1
    ) 

In [ ]:
estimator.train(input_fn = train,steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\erika\AppData\Local\Temp\tmpc8xmdbyk\model.ckpt.
INFO:tensorflow:loss = 277.20154, step = 1
INFO:tensorflow:global_step/sec: 2.41027
INFO:tensorflow:loss = 14.281899, step = 101 (41.491 sec)
INFO:tensorflow:global_step/sec: 2.53341
INFO:tensorflow:loss = 1.5989205, step = 201 (39.471 sec)
INFO:tensorflow:global_step/sec: 2.47086
INFO:tensorflow:loss = 0.13590361, step = 301 (40.502 sec)
INFO:tensorflow:global_step/sec: 2.28572
INFO:tensorflow:loss = 0.08037007, step = 401 (43.722 sec)
INFO:tensorflow:global_step/sec: 2.45711
INFO:tensorflow:loss = 0.0974065, step = 501 (40.699 sec)
INFO:tensorflow:global_step/sec: 2.47238
INFO:tensorflow:loss = 0.06883453, step = 601 (40.477 sec)
INFO:tensorflow:global_step/sec: 2.45374
INFO:tensorflow:loss = 0.052152853, step = 701 (40.722 sec)
INFO:tensorflow:global_step/sec: 2.38999
INFO:tensorflow:loss = 0.041176103, step = 801 (41.840 sec)
INFO:tenso

In [6]:
# defining the test inputs
input_fn2 = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test, 
    shuffle=False,
    batch_size=X_test.shape[0],
    num_epochs=None)

In [7]:
#evaluate the estimator
estimator.evaluate(input_fn2,steps=1000) 

INFO:tensorflow:Starting evaluation at 2018-12-04-09:48:18
INFO:tensorflow:Restoring parameters from C:\Users\erika\AppData\Local\Temp\tmpc8xmdbyk\model.ckpt-1000
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1000]
INFO:tensorflow:Evaluation [400/1000]
INFO:tensorflow:Evaluation [500/1000]
INFO:tensorflow:Evaluation [600/1000]
INFO:tensorflow:Evaluation [700/1000]
INFO:tensorflow:Evaluation [800/1000]
INFO:tensorflow:Evaluation [900/1000]
INFO:tensorflow:Evaluation [1000/1000]
INFO:tensorflow:Finished evaluation at 2018-12-04-09:51:12
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.99378884, average_loss = 0.0069061187, global_step = 1000, loss = 1.1118852


{'accuracy': 0.99378884,
 'average_loss': 0.0069061187,
 'global_step': 1000,
 'loss': 1.1118852}

In [8]:
#predict input
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    shuffle=False)

In [9]:
#predict the results
pred_results = estimator.predict(input_fn=pred_input_fn)

In [10]:
#print the results, example of the result is array([b'2'], dtype=object) 
#b = 2 = KIRC
for i in pred_results:
    print (i)

INFO:tensorflow:Restoring parameters from C:\Users\erika\AppData\Local\Temp\tmpc8xmdbyk\model.ckpt-1000
{'logits': array([11.072955  , -4.491851  , -0.74790883, -4.201838  , -5.5364833 ],
      dtype=float32), 'probabilities': array([9.9999213e-01, 1.7389506e-07, 7.3495494e-06, 2.3240099e-07,
       6.1179982e-08], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.5981465, 12.531471 , -6.984312 ,  1.2664882, -3.281811 ],
      dtype=float32), 'probabilities': array([1.9855142e-06, 9.9998510e-01, 3.3450032e-09, 1.2813669e-05,
       1.3563520e-07], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-1.2519577 , -2.9755962 ,  0.9142972 , -0.87970257,  5.904011  ],
      dtype=float32), 'probabilities': array([7.7334233e-04, 1.3797649e-04, 6.7478339e-03, 1.1221224e-03,
       9.9121875e-01], dtype=float32), 'class_ids': array([4], dtype=int64), 'classes': array([b'

In [11]:
y_test

array([0, 1, 4, 0, 1, 1, 1, 1, 0, 1, 3, 1, 1, 1, 0, 2, 1, 1, 3, 1, 3, 3,
       1, 1, 4, 1, 3, 1, 2, 3, 3, 4, 1, 1, 3, 4, 3, 4, 0, 0, 1, 4, 1, 3,
       4, 2, 1, 0, 1, 4, 0, 1, 4, 1, 1, 2, 3, 1, 0, 1, 1, 1, 0, 2, 2, 1,
       1, 0, 0, 4, 2, 2, 3, 1, 3, 3, 1, 0, 2, 2, 4, 2, 2, 0, 0, 2, 0, 1,
       1, 3, 0, 3, 2, 2, 3, 4, 0, 4, 2, 1, 4, 3, 2, 0, 2, 2, 0, 2, 1, 2,
       2, 2, 2, 0, 3, 1, 1, 3, 1, 1, 3, 3, 1, 0, 1, 3, 1, 2, 0, 2, 4, 2,
       1, 1, 2, 1, 1, 0, 2, 2, 3, 1, 3, 1, 1, 1, 3, 2, 1, 2, 0, 1, 1, 4,
       3, 0, 1, 2, 1, 1, 1], dtype=int64)

### References 
#### The code was obtained by using sklearn

##### https://www.tensorflow.org/api_docs/python/tf/contrib/learn/DNNClassifier

### References using vancouver
#### 1. tf.contrib.learn.DNNClassifier  |  TensorFlow [Internet]. TensorFlow. [cited 4 December 2018]. Available from: https://www.tensorflow.org/api_docs/python/tf/contrib/learn/DNNClassifier